# Question 2 Workflow


## Workfolow Summary

## Imports

In [1]:
# Import pandas and biothings explorers modules
import pandas as pandas
from biothings_explorer.query.predict import Predict
from biothings_explorer.query.visualize import display_graph
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore") 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Set Parameters

In [2]:
max_direct = 20
max_indirect = 50

In [3]:
ht = Hint()


{'Gene': [],
 'SequenceVariant': [],
 'ChemicalSubstance': [],
 'Disease': [],
 'PhenotypicFeature': [],
 'MolecularActivity': [],
 'BiologicalProcess': [],
 'CellularComponent': [],
 'Pathway': [{'WIKIPATHWAYS': 'WP396',
   'name': 'ACE Inhibitor Pathway',
   'primary': {'identifier': 'WIKIPATHWAYS',
    'cls': 'Pathway',
    'value': 'WP396'},
   'display': 'WIKIPATHWAYS(WP396) name(ACE Inhibitor Pathway)',
   'type': 'Pathway'},
  {'WIKIPATHWAYS': 'WP554',
   'name': 'ACE Inhibitor Pathway',
   'primary': {'identifier': 'WIKIPATHWAYS',
    'cls': 'Pathway',
    'value': 'WP554'},
   'display': 'WIKIPATHWAYS(WP554) name(ACE Inhibitor Pathway)',
   'type': 'Pathway'},
  {'PHARMGKB': 'PA2023',
   'name': 'ACE Inhibitor Pathway, Pharmacodynamics',
   'primary': {'identifier': 'PHARMGKB', 'cls': 'Pathway', 'value': 'PA2023'},
   'display': 'PHARMGKB(PA2023) name(ACE Inhibitor Pathway, Pharmacodynamics)',
   'type': 'Pathway'}],
 'AnatomicalEntity': [],
 'Cell': []}

In [3]:
disease_name = 'COVID-19'

disease = ht.query(disease_name)['Disease'][0]
disease

{'MONDO': 'MONDO:0100096',
 'DOID': 'DOID:0080600',
 'name': 'COVID-19',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0100096'},
 'display': 'MONDO(MONDO:0100096) DOID(DOID:0080600) name(COVID-19)',
 'type': 'Disease'}

In [4]:
# HYA = ht.query('hyaluronic acid')['ChemicalSubstance'][0]
# pd2 = Predict(
#     input_objs=[HYA],
#     intermediate_nodes =[None], 
#     output_types =['PhenotypicFeature'], 
#     config= {'expand' = }
# )
# pd2.connect(verbose=True)
# df = pd2.display_table_view()
# df

SyntaxError: invalid syntax (<ipython-input-4-960ca4e779c4>, line 6)

In [ ]:
df

## Get Symptoms

In [5]:
import requests 

# symptoms = ([ 'fever','cough', 'hemoptysis', 
# 'shortness of breath', 'myalgia', 'fatigue', 'sore throat', 
# 'nausea', 'vomiting', 'diarrhea', 'conjunctivitis', 'anorexia',
# 'headache', 'leukopenia', 'eosinopenia', 'neutrophilia', 'elevated liver enzymes', 'C-reactive protein', 'ferritin',
# 'hyaline membrane', 'alveolar lesions', 'prominent hyaline membranes', 'alveolar damage',
#  'acute myocardial injury', ' myocarditis', 'arrhythmias','cardiac dysfunction', 'encephalopathy'])

symptoms = (['cough', 'hemoptysis', 
'shortness of breath',
'hyaline membrane', 'alveolar lesions', 'prominent hyaline membranes', 'alveolar damage'])

symptom_dict = {}

for symptom_name in symptoms: 
    try: 
        symptom = ht.query(symptom_name)['PhenotypicFeature'][0]
#         print(symptom['name'])
#         pd = Predict(
#             input_objs=[symptom],
#             intermediate_nodes =[None], 
#             output_types = ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
#                 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
#                 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'], 
#             config={"expand": False}
#         )
#         pd.connect(verbose=False)
#         df = pd.display_table_view()
#         edges_out = df.shape[0]
        edges_out = 10
        symptom_dict[symptom['HP']] = {
            'names': [symptom['name'].upper()],
            'frequency': 'Unknown', 
            'edges_out' : edges_out
        }
        r = requests.get('https://biothings.ncats.io/hpo/phenotype/' + str(symptom['HP']))
        res = r.json()
#         print(res)
        if('synonym' in res):
            if('exact' in res['synonym']):
                for name in res['synonym']['exact']:
                    if name.upper() not in symptom_dict[res['_id']]['names']: 
#                         print(name.upper())
                        symptom_dict[res['_id']]['names'].append(name.upper())
    except:
        print(symptom_name + " failed")


hyaline membrane failed
alveolar lesions failed
prominent hyaline membranes failed


In [6]:
# symptom_dict
disease_symptom_df = pandas.DataFrame.from_dict(symptom_dict, orient='index')
disease_symptom_df

,names,frequency,edges_out
HP:0012735,"[COUGH, COUGHING]",Unknown,10
HP:0002105,"[HEMOPTYSIS, COUGHING UP BLOOD, HAEMOPTYSIS]",Unknown,10
HP:0002098,"[RESPIRATORY DISTRESS, BREATHING DIFFICULTIES,...",Unknown,10
HP:0033006,[DIFFUSE ALVEOLAR DAMAGE],Unknown,10


In [7]:
## also save disease symptoms as a list
disease_symptoms = []
for x in disease_symptom_df['names']: 
    disease_symptoms = disease_symptoms + x
# disease_symptoms

## Disease -> Gene -> [Phenotypic Feature, Disease, Biological Process]

In [8]:
pd = Predict(
    input_objs=[disease],
    intermediate_nodes =['Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config={"expand": False}
)
pd.connect(verbose=False)

API call to BioLink API with input 4141 failed with timeout error. Current timeout limit is 5 seconds
API call to CORD Gene API with input ['5417', '2367', '10618', '13523', '9281', '30615', '5438', '16885', '8568', '11947', '4989', '5962', '6018', '6001', '5414', '1706', '5992', '13557', '6898', '17697', '6008', '3535', '338', '5981', '6915', '27954', '11183', '3009', '2707', '1678', '11876', '9958', '399', '10637', '1116', '19679', '16884', '11782', '2524', '2537', '9208', '5434', '6741', '11892', '6025', '27960', '79', '195', '4113', '6023', '336', '3541', '10627', '4552'] failed with timeout error. Current timeout limit is 20 seconds
API call to SEMMED Gene API with input ['C1413809', 'C1336641', 'C0054871', 'C1412332', 'C1412186', 'C1333916', 'C1420310', 'C0879590', 'C1415274', 'C1426329', 'C1412828', 'C1332690', 'C1332714', 'C1413244', 'C1420826', 'C1332807', 'C1334098', 'C1540024', 'C1439284', 'C1334114', 'C1417035', 'C1335817', 'C1335818', 'C1413931', 'C1367714', 'C1412099', 'C

In [9]:
df = pd.display_table_view()
df.shape

(32424, 19)

In [10]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df['output_label'])]) if val in disease_symptoms]
direct_relevant_genes_to_symptoms_df = df.iloc[indices_with_symptom_outputs]
direct_relevant_genes_to_symptoms_df.shape

(6, 19)

In [11]:
direct_genes = 	list(direct_relevant_genes_to_symptoms_df["node1_label"])
direct_gene_dict = {x:direct_genes.count(x) for x in direct_genes}
direct_gene_dict = dict(sorted(direct_gene_dict.items(), key=lambda x: x[1], reverse=True)[:max_direct])
direct_gene_dict

{'ACE': 3, 'MARS1': 1, 'TNF': 1, 'AGTR1': 1}

## Disease X -> Intermediate Node -> Gene ->  [Phenotypic Feature, Disease, Biological Process]

In [12]:
query_config = {
    "filters": [
        {
        },
        {
            "nodeDegree": {
                ">" : 20
            }
        }
    ],
    "annotate": ["ngd","nodeDegree"],
    "expand": False
}

In [13]:
## way to filter at the gene step for top x number of genes? 
pd2 = Predict(
    input_objs=[disease],
    intermediate_nodes =[['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'],'Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config= query_config 
)
pd2.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

Your query have 1 input nodes, including COVID-19 .... And BTE will find paths that connect your input nodes to your output types ['PhenotypicFeature', 'Disease', 'BiologicalProcess']. Paths will contain 2 intermediate nodes.

Intermediate node #1 will have these type constraints: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Intermediate node #2 will have these type constraints: Gene

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Disease
Output Types: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Predicates: None

BTE found 12 APIs based on SmartAPI Meta-KG.

API 1. Ontology Lookup Service API (1 API calls)
API 2. Automat CORD19 Scibite API (3 AP

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API 13.2 Text Mining Targeted Association API: 0 hits
API 16.3 CTD API: 67 hits
API 16.4 CTD API: 127 hits
API 11.1: https://mygene.info/v3/query?fields=pantherdb.ortholog (POST -d q=340024,54938,92521,925,7054,3558,22848,4151,10410,5045,6868,2147,3553,339967,4141,5499,1803,3574,682,28,5972,2152,10473,1401,3456,10045,6347,59272,213,2875,7124,3439,1506,5447,6651,10044,7137,2580,3576,54585,1514,1636,3627,3559,3458,6348,3586,3569,185,920,186,7113,3605,10332&scopes=entrezgene)
API 11.1 MyGene.info API: 82 hits
API 7.1: https://api.monarchinitiative.org/api/bioentity/disease/MONDO:0020753/genes?direct=True&rows=200&unselect_evidence=True
API 7.1 BioLink API: 0 hits
API 7.2: https://api.monarchinitiative.org/api/bioentity/disease/MONDO:0020753/genes?direct=True&rows=200&unselect_evidence=True
API 7.2 BioLink API: 0 hits
API 15.1: https://mychem.info/v1/query?fields=drugbank.targets (POST -d q=DB09140,DB00503,DB01601,DB13609,DB00148,DB00608,DB01611&scopes=drugbank.id)
API 15.1 MyChem.info API

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API 12.4 SEMMED Gene API: 2 hits


...

API 8.209 BioLink API: 13 hits
API 6.209 Automat PHAROS API: 42 hits
API 1.209 Automat CORD19 Scibite API: 36 hits
API 8.210: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6347/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.210 BioLink API: 0 hits
API 8.211: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:54106/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.211 BioLink API: 0 hits
API 1.210: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:1544
API 1.210 Automat CORD19 Scibite API: 0 hits
API 1.211: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:54106
API 1.211 Automat CORD19 Scibite API: 13 hits
API 8.212: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:146/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.212 BioLink API: 0 hits
API 1.212: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:146
API 1.212 Automat CORD19 Scibite API: 0 hits
API 8.213: https://api.

API 8.226 BioLink API: 0 hits
API 8.227: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3976/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.227 BioLink API: 0 hits
API 6.225: https://automat.renci.org/pharos/gene/disease/NCBIGene:7098
API 6.225 Automat PHAROS API: 2 hits
API 1.225: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:351
API 3.225: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:5743
API 3.225 Automat CORD19 Scigraph API: 0 hits
API 6.226: https://automat.renci.org/pharos/gene/disease/NCBIGene:6774
API 6.226 Automat PHAROS API: 2 hits
API 6.227: https://automat.renci.org/pharos/gene/disease/NCBIGene:5743
API 6.227 Automat PHAROS API: 3 hits
API 1.225 Automat CORD19 Scibite API: 27 hits
API 1.226: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6774
API 1.226 Automat CORD19 Scibite API: 31 hits
API 1.227: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:3976
API 1.227 Automat CORD19 Scib

API 3.244 Automat CORD19 Scigraph API: 1 hits
API 1.244 Automat CORD19 Scibite API: 39 hits
API 8.242: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1509/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.242 BioLink API: 23 hits
API 8.243: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7040/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.243: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1161/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.243 BioLink API: 103 hits
API 8.244 BioLink API: 81 hits
API 8.245: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3133/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.245 BioLink API: 0 hits
API 1.245: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:1442
API 1.245 Automat CORD19 Scibite API: 0 hits
API 8.246: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3497/phenotypes?direct=True&rows=200&unselect_evidence=

API 8.260 BioLink API: 0 hits
API 8.261: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1234/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.261 BioLink API: 0 hits
API 8.262: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:940/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.262 BioLink API: 1 hits
API 6.260: https://automat.renci.org/pharos/gene/disease/NCBIGene:5781
API 6.260 Automat PHAROS API: 4 hits
API 6.261: https://automat.renci.org/pharos/gene/disease/NCBIGene:27352
API 6.261 Automat PHAROS API: 1 hits
API 6.262: https://automat.renci.org/pharos/gene/disease/NCBIGene:7132
API 6.262 Automat PHAROS API: 2 hits
API 1.260: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:27352
API 1.260 Automat CORD19 Scibite API: 1 hits
API 1.261: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:5781
API 1.261 Automat CORD19 Scibite API: 1 hits
API 1.262: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGe

API 3.278 Automat CORD19 Scigraph API: 2 hits
API 3.279: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:3661
API 3.279 Automat CORD19 Scigraph API: 26 hits
API 8.277: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4015/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.277: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:26073/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.277 BioLink API: 0 hits
API 8.278 BioLink API: 49 hits
API 8.279: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:100/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.279 BioLink API: 67 hits
API 8.280: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6850/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.280 BioLink API: 0 hits
API 1.280: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:4615
API 1.280 Automat CORD19 Scibite API: 13 hits
API 1.281: https://automat.renci.org/cord19-scib

In [14]:
df2 = pd2.display_table_view()
df2.shape

(8339046, 27)

In [15]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df2['output_label'])]) if val in disease_symptoms]
indirect_relevant_genes_to_symptoms_df = df2.iloc[indices_with_symptom_outputs]
indirect_relevant_genes_to_symptoms_df.shape


(1310, 27)

In [16]:
list(dict.fromkeys(list(indirect_relevant_genes_to_symptoms_df['node2_label'])))

['UMLS:C0033551',
 'UMLS:C1705556',
 'UMLS:C0014442',
 'UMLS:C0017337',
 'AGTR1',
 'INS',
 'ACE',
 'UMLS:C1705543',
 'UMLS:C0030956',
 'FOS',
 'TNF',
 'UMLS:C0033640',
 'UMLS:C0132555',
 'GRK2',
 'STAT3',
 'UMLS:C0085940',
 'IGHE',
 'MMP9',
 'VEGFA',
 'C5AR1',
 'UMLS:C1705744',
 'LOX',
 'CD79A']

In [17]:
indirect_genes = 	list(indirect_relevant_genes_to_symptoms_df["node2_label"])
indirect_gene_dict = {x:indirect_genes.count(x) for x in indirect_genes}
indirect_gene_dict = dict(sorted(indirect_gene_dict.items(), key=lambda x: x[1], reverse=True)[:100])
indirect_gene_dict

{'VEGFA': 166,
 'TNF': 146,
 'ACE': 117,
 'UMLS:C0017337': 111,
 'UMLS:C0014442': 83,
 'INS': 69,
 'UMLS:C0030956': 63,
 'STAT3': 60,
 'FOS': 53,
 'MMP9': 49,
 'UMLS:C1705556': 45,
 'UMLS:C0033640': 39,
 'AGTR1': 33,
 'UMLS:C1705744': 33,
 'IGHE': 32,
 'UMLS:C0033551': 29,
 'UMLS:C1705543': 29,
 'UMLS:C0132555': 29,
 'GRK2': 27,
 'UMLS:C0085940': 27,
 'LOX': 24,
 'C5AR1': 23,
 'CD79A': 23}

## Top Genes -> Pathways 

In [18]:
# get gene inputs through hint module
gene_inputs = []
gene_names = list(direct_gene_dict.keys()) + list(indirect_gene_dict.keys())
gene_names = [x for x in gene_names if 'UMLS' not in x ]
gene_names = list(dict.fromkeys(gene_names))
gene_names
for gene in gene_names: 
    try: 
        gene_input = ht.query(gene)["Gene"][0]
        gene_inputs.append(gene_input)
    except: 
        print(gene + ' Failed')

# print(gene_inputs)

In [19]:
# pd3 = Predict(
#     input_objs=gene_inputs,
#     intermediate_nodes=[None], 
#     output_types =['Pathway'], 
#     config={"expand": False}
# )
# pd3.connect(verbose=True)

In [20]:
# df3 = pd3.display_table_view()
# df3.shape

In [21]:
# predict_many funciton will be used to run many BTE queries and return results as a single table
def predict_many(input_object_list, output_type_list, intermediate_node_list = ''):
    df_list = []
    for input_object in input_object_list: 
        for output_type in output_type_list: 
            if(len(intermediate_node_list) > 0):
                for inter in intermediate_node_list:
                    try: 
                        print("Running: " + input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type )
                        fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=[inter])
                        fc.connect(verbose=False)
                        df = fc.display_table_view()
                        rows = df.shape[0]
                        if(rows > 0):
                            df_list.append(df)
                    except:
                        print(input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type + ' FAILED')
            else:
                try:
                    print("Running: " + input_object['name'] + ' --> output type ' + output_type )
                    fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=None)
                    fc.connect(verbose=False)
                    df = fc.display_table_view()
                    rows = df.shape[0]
                    if(rows > 0):
                        df_list.append(df)
                except:
                    print(input_object['name'] + ' --> output type ' + output_type + ' FAILED')

    if(len(df_list) > 0):
        return pandas.concat(df_list)
    else:
        return None

In [22]:
gene_to_pathways = predict_many(gene_inputs, ['Pathway'])
list(dict.fromkeys(list(gene_to_pathways["output_name"])))

Running: angiotensin I converting enzyme --> output type Pathway
Running: methionyl-tRNA synthetase 1 --> output type Pathway
Running: tumor necrosis factor --> output type Pathway
Running: angiotensin II receptor type 1 --> output type Pathway
Running: vascular endothelial growth factor A --> output type Pathway
Running: insulin --> output type Pathway
Running: signal transducer and activator of transcription 3 --> output type Pathway
Running: Fos proto-oncogene, AP-1 transcription factor subunit --> output type Pathway
Running: matrix metallopeptidase 9 --> output type Pathway
Running: immunoglobulin heavy constant epsilon --> output type Pathway
Running: G protein-coupled receptor kinase 2 --> output type Pathway
Running: lysyl oxidase --> output type Pathway
Running: complement C5a receptor 1 --> output type Pathway
Running: CD79a molecule --> output type Pathway


['METABOLISM OF ANGIOTENSINOGEN TO ANGIOTENSINS',
 'PEPTIDE HORMONE METABOLISM',
 'METABOLISM OF PROTEINS',
 'ACE INHIBITOR PATHWAY',
 'METABOLISM',
 'SELENOAMINO ACID METABOLISM',
 'CYTOSOLIC TRNA AMINOACYLATION',
 'TRNA AMINOACYLATION',
 'METABOLISM OF AMINO ACIDS AND DERIVATIVES',
 'TRANSLATION',
 'DEVELOPMENTAL BIOLOGY',
 'CYTOKINE SIGNALING IN IMMUNE SYSTEM',
 'SIGNAL TRANSDUCTION',
 'IMMUNE SYSTEM',
 'TRANSCRIPTIONAL REGULATION OF WHITE ADIPOCYTE DIFFERENTIATION',
 'SIGNALING BY INTERLEUKINS',
 'TNFR1-INDUCED PROAPOPTOTIC SIGNALING',
 'REGULATION OF TNFR1 SIGNALING',
 'TNFR1-INDUCED NFKAPPAB SIGNALING PATHWAY',
 'TNFR1-MEDIATED CERAMIDE PRODUCTION',
 'TNFR2 NON-CANONICAL NF-KB PATHWAY',
 'INTERLEUKIN-10 SIGNALING',
 'INTERLEUKIN-4 AND INTERLEUKIN-13 SIGNALING',
 'DEATH RECEPTOR SIGNALLING',
 'TNF SIGNALING',
 'MATRIX METALLOPROTEINASES',
 'REGULATION OF TOLL-LIKE RECEPTOR SIGNALING PATHWAY',
 'SELENIUM MICRONUTRIENT NETWORK',
 'VITAMIN B12 METABOLISM',
 'MICRORNAS IN CARDIOMYOCYT

In [23]:
# pathways
gene_to_pathway_results = {}
gene_to_pathway_genes = list(gene_to_pathways["output_name"]) # create list of genes
gene_to_pathway_genes = list(dict.fromkeys(gene_to_pathway_genes))  # remove duplicates

for gene in gene_to_pathway_genes: 
    gene_to_pathway_results[gene] = {
        'pathway_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_pathways.iterrows():
    gene_to_pathway_results[row['output_name']]['pathway_count'] = gene_to_pathway_results[row['output_name']]['pathway_count'] + 1
    gene_to_pathway_results[row['output_name']]['genes_related'].append(row['input'])
    

gene_to_pathway_results = dict(sorted(gene_to_pathway_results.items(), key = lambda x: x[1]['pathway_count'], reverse = True))

    
# gene_to_pathway_results
pandas.DataFrame.from_dict(gene_to_pathway_results, orient='index').iloc[0:50]

,pathway_count,genes_related
SIGNAL TRANSDUCTION,9,"[TNF, AGTR1, VEGFA, INS, STAT3, FOS, MMP9, GRK..."
IMMUNE SYSTEM,7,"[TNF, VEGFA, STAT3, FOS, MMP9, C5AR1, CD79A]"
INTERLEUKIN-4 AND INTERLEUKIN-13 SIGNALING,6,"[TNF, VEGFA, STAT3, FOS, MMP9, IGHE]"
CYTOKINE SIGNALING IN IMMUNE SYSTEM,5,"[TNF, VEGFA, STAT3, FOS, MMP9]"
SIGNALING BY INTERLEUKINS,5,"[TNF, VEGFA, STAT3, FOS, MMP9]"
SIGNALING BY RECEPTOR TYROSINE KINASES,5,"[VEGFA, INS, STAT3, FOS, MMP9]"
METABOLISM OF PROTEINS,4,"[ACE, MARS1, INS, STAT3]"
DEVELOPMENTAL BIOLOGY,4,"[TNF, INS, STAT3, MMP9]"
ALLOGRAFT REJECTION,3,"[TNF, AGTR1, VEGFA]"
ADIPOGENESIS,3,"[TNF, INS, STAT3]"


## Genes --> Bioprocesses

In [24]:
# bioprocesss
gene_to_bioprocesses = predict_many(gene_inputs, ['BiologicalProcess'])
gene_to_bioprocess_results = {}
gene_to_bioprocess_genes = list(gene_to_bioprocesses["output_name"]) # create list of genes
gene_to_bioprocess_genes = list(dict.fromkeys(gene_to_bioprocess_genes))  # remove duplicates

for gene in gene_to_bioprocess_genes: 
    gene_to_bioprocess_results[gene] = {
        'bioprocess_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_bioprocesses.iterrows():
    gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] = gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] + 1
    gene_to_bioprocess_results[row['output_name']]['genes_related'].append(row['input'])


Running: angiotensin I converting enzyme --> output type BiologicalProcess
Running: methionyl-tRNA synthetase 1 --> output type BiologicalProcess
Running: tumor necrosis factor --> output type BiologicalProcess
Running: angiotensin II receptor type 1 --> output type BiologicalProcess
Running: vascular endothelial growth factor A --> output type BiologicalProcess
Running: insulin --> output type BiologicalProcess
Running: signal transducer and activator of transcription 3 --> output type BiologicalProcess
Running: Fos proto-oncogene, AP-1 transcription factor subunit --> output type BiologicalProcess
Running: matrix metallopeptidase 9 --> output type BiologicalProcess
Running: immunoglobulin heavy constant epsilon --> output type BiologicalProcess
Running: G protein-coupled receptor kinase 2 --> output type BiologicalProcess
Running: lysyl oxidase --> output type BiologicalProcess
Running: complement C5a receptor 1 --> output type BiologicalProcess
Running: CD79a molecule --> output typ

In [25]:
gene_to_bioprocess_results = dict(sorted(gene_to_bioprocess_results.items(), key = lambda x: x[1]['bioprocess_count'], reverse = True))

In [26]:
counter = 0 
for key in gene_to_bioprocess_results.keys(): 
    if counter < 100: 
        if(('C0' in key) or ('C1' in key)): 
            try: 
                name = ht.query(key)['BiologicalProcess'][0]['name']
                gene_to_bioprocess_results[name] = gene_to_bioprocess_results[key]
                del gene_to_bioprocess_results[key]
            except: 
                pass
    counter = counter + 1

RuntimeError: dictionary keys changed during iteration

In [ ]:
# gene_to_bioprocess_results
pandas.DataFrame.from_dict(gene_to_bioprocess_results, orient='index').iloc[0:50]